In [1]:
from util import (get_postgresql_conn,
                  get_sql_alchemy_engine, 
                    SQL_ALCHEMY_CONN_STRING, 
                    DB_NAME, DB_USER, 
                    DB_PASSWORD, 
                    DB_PORT, 
                    DB_HOST)
import pandas as pd
import cufflinks as cf
import numpy as np
import plotly as py
import seaborn as sns
import plotly.express as px
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
conn = get_sql_alchemy_engine(SQL_ALCHEMY_CONN_STRING)

In [35]:

df = pd.read_sql("SELECT * FROM listing;", conn)

In [36]:
df.head(3)

,mls_num,address_street,num_of_beds,num_of_baths,sqft,property_type,price,last_updated,link,status,city,province,postal_code
0,1249295,369 Old Pennywell Road,0,0,0,Vacant Land,230000,2022-09-02,https://www.remax.ca/nl/st-john-s-real-estate/...,Active,St. John's,NL,A1B 1A8
1,1251754,6-8 Dakota Place,0,0,0,Vacant Land,74000,2023-01-19,https://www.remax.ca/nl/paradise-real-estate/6...,Active,Paradise,NL,A1L 3G3
2,1249614,101 Springdale Street,0,0,0,Vacant Land,110000,2023-04-18,https://www.remax.ca/nl/st-john-s-real-estate/...,Active,St. John's,NL,A1C 5B7


In [47]:
# df.query('city == "St. Johns"')
# df.loc[df['city'] == "St.John\'s",'city'] = "St. John\'s"
# df.loc[df['city'] == "St. Johns",'city'] = "St. John\'s"

In [5]:
# Line plot
import plotly.graph_objects as go

In [51]:
px.histogram(data_frame=df,
        x='city',
        labels={'x':'city', 'y':'count'})
px.histogram(data_frame=df,
                x='property_type',
                labels={'x':'property_type', 'y':'count'})
